In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import six

sns.set(style='ticks', context='paper', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian'
!mkdir $folder
import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian': File exists


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (11,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-02-23 09:18:48	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-02-23 09:18:48	Parsing datapackage to create a Study object
2016-02-23 09:19:21	Initializing Study
2016-02-23 09:19:21	Initializing Predictor configuration manager for Study
2016-02-23 09:19:21	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-02-23 09:19:21	Added ExtraTreesClassifier to default predictors
2016-02-23 09:19:21	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-02-23 09:19:21	Added ExtraTreesRegressor to default predictors
2016-02-23 09:19:21	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-02-23 09:19:21	Added GradientBoostingClassifier to default predictors
2016-02-23 09:19:21	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [2]:
figure_folder = '{}/intron_conservation'.format(folder)
! mkdir $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/intron_conservation': File exists


In [3]:
ls /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2*csv

ls: cannot access /projects/ps-yeolab/obotvinnik/singlecell_pnms/exon2*csv: No such file or directory


In [4]:
csv_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper'

In [5]:
splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

In [6]:
alternative_feature_folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative'

In [7]:
downstream = pd.read_csv('{}/downstream400_placental_mammal_conservation.csv'.format(alternative_feature_folder), 
                         index_col=0)
upstream = pd.read_csv('{}/upstream400_placental_mammal_conservation.csv'.format(alternative_feature_folder), 
                       index_col=0)

In [8]:
conservation_folder = '{}/conservation'.format(folder)

In [9]:
from anchor import MODALITY_ORDER, MODALITY_PALETTE

In [10]:
sns.set(style='ticks', context='paper')

In [11]:
from anchor import MODALITY_ORDER

In [12]:
constitutive_downstream = pd.read_csv('{}/downstream400_placental_mammal_conservation.csv'.format(constitutive_feature_folder), 
                         index_col=0)
constitutive_upstream = pd.read_csv('{}/upstream400_placental_mammal_conservation.csv'.format(constitutive_feature_folder), 
                       index_col=0)

In [13]:
constitutive_conservations = {'Upstream': constitutive_upstream, 'Downstream': constitutive_downstream}

In [14]:
from scipy.stats import mannwhitneyu

In [15]:
# %%time

intron_conservations = {'Upstream': upstream, 
                        'Downstream': downstream}

modality_basewise_conservation_dfs = []

intron_order = ['Upstream', 'Downstream']

values_col = r'$\mathrm{sgn}(\Delta\mathrm{mean}) \times\left(-\log_{10}(q)\right)$'


phenotype_basewise_statistics_dfs = []

for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    modalities = phenotype_df.set_index('event_id').modality
    modalities = modalities.drop(modalities.index.get_duplicates())
    
    conservation_dfs = []
    basewise_statistics_dfs = []
    

    for intron_name, intron_conservation in intron_conservations.items():
        mean_conservation = intron_conservation.groupby(modalities).mean()
        mean_conservation.columns = mean_conservation.columns.astype(int) + 1
        tidy = mean_conservation.unstack().reset_index()
        tidy = tidy.rename(columns={'level_0':'genome_direction_nt', 0: 'Conservation'})
        tidy['direction'] = intron_name
        conservation_dfs.append(tidy)
        conservation_grouped = intron_conservation.groupby(modalities)
#         constitutive_name = '~1'

#         constitutive = conservation_grouped.get_group(constitutive_name)
        constitutive = constitutive_conservations[intron_name]
    
        constitutive_mean = constitutive.mean()
        constitutive_mean.index = constitutive_mean.index.astype(int) + 1
        tidy = constitutive_mean.reset_index()
        tidy['modality'] = 'constitutive'
        tidy['direction'] = intron_name
        tidy = tidy.rename(columns={'index':'genome_direction_nt', 0: 'Conservation'})
        conservation_dfs.append(tidy)
        
        for modality, df in conservation_grouped:
            if modality == constitutive_name:
                continue
            lines = []

            for i in df:
                statistic, p_value = mannwhitneyu(df[i], constitutive[i])
                sign = np.sign(df[i].mean() - constitutive[i].mean())
                lines.append([statistic, p_value, sign])
            intron_basewise_statistics = pd.DataFrame(lines, columns=['U Statistic', '$p$-value', 'sign'])
            intron_basewise_statistics['$q$-value'] = intron_basewise_statistics.shape[0] * intron_basewise_statistics['$p$-value']
            intron_basewise_statistics['modality'] = modality
            intron_basewise_statistics['intron'] = intron_name
            intron_basewise_statistics = intron_basewise_statistics.reset_index()
            intron_basewise_statistics = intron_basewise_statistics.rename(columns={'index': 'genome_direction_nt'})
#             print '\n', phenotype, intron_name, modality
#             print intron_basewise_statistics.groupby('genome_direction_nt').size().head()
            basewise_statistics_dfs.append(intron_basewise_statistics)

    basewise_statistics = pd.concat(basewise_statistics_dfs, ignore_index=True)
    basewise_statistics['$-\log_{10}(q)$'] = -np.log10(basewise_statistics['$q$-value'])
    basewise_statistics['$-\log_{10}(p)$'] = -np.log10(basewise_statistics['$p$-value'])

    basewise_statistics[values_col] = basewise_statistics['sign'] * basewise_statistics['$-\log_{10}(q)$']
    basewise_statistics['phenotype'] = phenotype
#     print basewise_statistics.groupby(['modality', 'intron', 'genome_direction_nt']).size().head()
#     print 'basewise_statistics.shape', basewise_statistics.shape
    phenotype_basewise_statistics_dfs.append(basewise_statistics)
#     basewise_statistics.head()

#         var_conservation = intron_conservation.groupby(modalities).var()
    modality_basewise_conservation = pd.concat(conservation_dfs, ignore_index=True)
    modality_basewise_conservation['phenotype'] = phenotype
    modality_basewise_conservation_dfs.append(modality_basewise_conservation)

modality_basewise_conservation = pd.concat(modality_basewise_conservation_dfs, ignore_index=True)
print modality_basewise_conservation.shape
modality_basewise_statistics = pd.concat(phenotype_basewise_statistics_dfs, ignore_index=True)
print modality_basewise_statistics.shape
modality_basewise_statistics.head()

NameError: name 'constitutive_name' is not defined

In [ ]:
import six

In [ ]:
phenotype_grouped = study.supplemental.modalities_tidy.groupby('phenotype')
ipsc = phenotype_grouped.get_group('iPSC')
ipsc_modalities = ipsc.groupby('modality')


for intron_name, intron_conservation in intron_conservations.items():
    six.print_(intron_name)
    for phenotype in study.phenotype_order[1:]:
        six.print_('\t', phenotype)
        phenotype_df = phenotype_grouped.get_group(phenotype)
        for modality, modality_df in phenotype_df.groupby('modality'):
            six.print_('\t\t', modality)
            ipsc = ipsc_modalities.get_group(modality)
            ipsc_conservation = intron_conservation.loc[ipsc['event_id']].mean(axis=1)
            try:
                phenotype_conservation = intron_conservation.loc[modality_df['event_id']].mean(axis=1)
            except KeyError:
                continue
            
            statistic, p_value = mannwhitneyu(ipsc_conservation, phenotype_conservation)
            six.print_('\t\t\t', statistic, p_value)

In [ ]:
modality_order = ['constitutive'] + MODALITY_ORDER[:3] 
modality_palette = ['k'] + MODALITY_PALETTE[:3]

In [ ]:
g = sns.FacetGrid(modality_basewise_conservation, hue='modality', hue_order=modality_order,
                 palette=modality_palette, row='phenotype', row_order=study.phenotype_order,
                  col='direction', sharex=False, #gridspec_kws=dict(wspace=0.1), #legend=False,
                  col_order=intron_order, size=2)
g.map(plt.plot, 'genome_direction_nt', 'Conservation')
for ax in g.axes.flat:
    ax.locator_params(nbins=4)
#         ax.locator_params('y', nbins=4)
    if ax.is_first_col():
        ax.set(ylabel='Mean PhastCons\n(Placental Mammal)',
               xticklabels=list(reversed([int(x) for x in ax.get_xticks()])))
    else:
        sns.despine(ax=ax, left=True)
        ax.yaxis.set_ticks_position('none')
g.set_titles('{row_name}')
g.set(xlabel='Nucleotides from exon', ylim=(0, 1))
# g.add_legend();
g.fig.subplots_adjust(wspace=0.01)
g.fig.tight_layout()
g.savefig('{}/intron_basewise_conservation_400nt.pdf'.format(figure_folder))

In [ ]:
g = sns.FacetGrid(modality_basewise_conservation, hue='modality', hue_order=modality_order,
                 palette=modality_palette, row='phenotype', row_order=study.phenotype_order,
                  col='direction', sharex=False, #gridspec_kws=dict(wspace=0.1), #legend=False,
                  col_order=intron_order, size=2, aspect=.75)
g.map(plt.plot, 'genome_direction_nt', 'Conservation')
for ax in g.axes.flat:
    ax.locator_params(nbins=4)
#         ax.locator_params('y', nbins=4)
    if ax.is_first_col():
        xticklabels = ax.get_xticks()
        ax.set(xlim=(200, 400))
        ax.set(ylabel='Mean PhastCons\n(Placental Mammal)',
               xticklabels=[200, 150, 100, 50, 0])
    else:
        sns.despine(ax=ax, left=True)
        ax.yaxis.set_ticks_position('none')
        ax.set(xlim=(0, 200))
g.set_titles('{row_name}')
g.set(xlabel='Nucleotides from exon', ylim=(0, 1))
# g.add_legend();
g.fig.subplots_adjust(wspace=0.01)
g.fig.tight_layout()
g.savefig('{}/intron_basewise_conservation_200nt.pdf'.format(figure_folder))

In [ ]:
# import matplotlib as mpl
# dark2 = sns.color_palette('Dark2', n_colors=10)
# sns.palplot(dark2)
# accent = sns.color_palette('Accent', n_colors=10)
# sns.palplot(accent)

# sns.palplot(sns.color_palette('PRGn', n_colors=2))
# direction_to_color = dict(zip(['upstream', 'downstream'], 
#                             map(mpl.colors.rgb2hex, sns.color_palette('PRGn', n_colors=2))))


# # This is the winner!!
# import matplotlib as mpl
# celltype_palette = dark2[0], accent[4], dark2[3], accent[2]
# sns.palplot(celltype_palette)

# celltype_palette = map(mpl.colors.rgb2hex, celltype_palette)

# phenotype_to_color = dict(zip(['iPSC', 'NPC', "MN", "sMN"], celltype_palette))
# phenotype_to_color

In [ ]:
g = sns.factorplot(col='direction', y='Conservation', hue='phenotype', size=2, col_order=intron_order,
               x='modality', data=modality_basewise_conservation, kind='bar', order=MODALITY_ORDER,
               hue_order=study.phenotype_order, palette=study.phenotype_color_ordered)
g.set_ylabels('Mean PhastCons\n(Placental Mammal)')
g.set(xlabel='')
for ax in g.axes.flat:
    plt.setp(ax.get_xticklabels(), rotation=90)
# g.fig.tight_layout()
g.savefig('{}/intron_mean_conservation.pdf'.format(figure_folder))

In [ ]:
cmap = sns.dark_palette(177, as_cmap=True)

In [ ]:
for phenotype, phenotype_df in modality_basewise_statistics.groupby('phenotype'):
    # PLot significance of basewise statistics
    figwidth = 2.5 * 2
    figheight = 0.3 * len(modality_basewise_statistics.groupby('modality').groups)

    fig, axes = plt.subplots(ncols=3, figsize=(figwidth, figheight), gridspec_kw=dict(width_ratios=[1, 1, 0.1], wspace))
    cbar_ax = axes[-1]

    grouped = phenotype_df.groupby('intron')

    for intron, ax in zip(intron_order, axes):
        df = grouped.get_group(intron)
        basewise_statistics_2d = df.pivot(index='modality', columns='genome_direction_nt', values='$-\log_{10}(q)$')
        basewise_statistics_2d = basewise_statistics_2d.loc[MODALITY_ORDER[:3]].dropna()
        sns.heatmap(basewise_statistics_2d, cmap=cmap, ax=ax, vmin=0,
                    cbar_kws=dict(label='$-\log_{10}(q)$'), 
                    cbar_ax=cbar_ax)
        ax.set(xticks=[])
        if not ax.is_first_col():
            ax.set(yticks=[], ylabel='')
        else:
            plt.setp(ax.get_yticklabels(), rotation=0)
    fig.subplots_adjust(wspace=0.01)
    fig.tight_layout()
    fig.savefig('{}/intron_basewise_conservation_statistics_{}_q_values.pdf'.format(figure_folder, phenotype))

In [ ]:
for phenotype, phenotype_df in modality_basewise_statistics.groupby('phenotype'):
    # PLot significance of basewise statistics
    figwidth = 2.5 * 2
    figheight = 0.3 * len(modality_basewise_statistics.groupby('modality').groups)

    fig, axes = plt.subplots(ncols=3, figsize=(figwidth, figheight), 
                             gridspec_kw=dict(width_ratios=[1, 1, 0.1], wspace=0.2))
    cbar_ax = axes[-1]

    grouped = phenotype_df.groupby('intron')

    for intron, ax in zip(intron_order, axes):
        df = grouped.get_group(intron)
        basewise_statistics_2d = df.pivot(index='modality', columns='genome_direction_nt', values='$-\log_{10}(q)$')
        basewise_statistics_2d = basewise_statistics_2d.loc[MODALITY_ORDER[:3]].dropna()
        sns.heatmap(basewise_statistics_2d, cmap=cmap, ax=ax, vmin=0, vmax=25,
                    cbar_kws=dict(label='$-\log_{10}(q)$'), 
                    cbar_ax=cbar_ax)
        ax.set(xticks=[])
        if not ax.is_first_col():
            ax.set(yticks=[], ylabel='', xlim=(0, 200))
        else:
            plt.setp(ax.get_yticklabels(), rotation=0)
            ax.set(xlim=(200, 400))
    fig.subplots_adjust(wspace=0.01)
    sns.despine(bottom=False, right=False, left=False, top=False)
    fig.tight_layout()
    fig.savefig('{}/intron_basewise_conservation_statistics_{}_q_values_200nt.pdf'.format(figure_folder, phenotype))

In [ ]:
for phenotype, phenotype_df in modality_basewise_statistics.groupby('phenotype'):
    # PLot significance of basewise statistics
    figwidth = 2.5 * 2
    figheight = 0.3 * len(modality_basewise_statistics.groupby('modality').groups)

    fig, axes = plt.subplots(ncols=3, figsize=(figwidth, figheight), gridspec_kw=dict(width_ratios=[1, 1, 0.1]))
    cbar_ax = axes[-1]

    grouped = phenotype_df.groupby('intron')

    for intron, ax in zip(intron_order, axes):
        df = grouped.get_group(intron)
        basewise_statistics_2d = df.pivot(index='modality', columns='genome_direction_nt', values='$-\log_{10}(p)$')
        basewise_statistics_2d = basewise_statistics_2d.loc[MODALITY_ORDER[:3]].dropna()
        sns.heatmap(basewise_statistics_2d, cmap=cmap, ax=ax,
                    cbar_kws=dict(label='$-\log_{10}(p)$'), 
                    cbar_ax=cbar_ax)
        ax.set(xticks=[])
        if not ax.is_first_col():
            ax.set(yticks=[], ylabel='')
        else:
            plt.setp(ax.get_yticklabels(), rotation=0)
    fig.subplots_adjust(wspace=0.01)
    fig.tight_layout()
    fig.savefig('{}/intron_basewise_conservation_statistics_{}_p_values.pdf'.format(figure_folder, phenotype))